# Correction to [10158 Issue](https://github.com/Qiskit/qiskit-terra/issues/10158) from qiskit-terra

In [ ]:
!pip install qiskit

In [ ]:
from qiskit.result import QuasiDistribution,ProbDistribution
dist_1 = {'01': 0.369, '00': 0.13975, '10': 0.45875, '11': 0.0325}
dist_2 = {'00': 0.011835180722095105,
 '01': 0.48784140401270437,
 '10': 0.4983486948083151,
 '11': 0.001974720456885419}
dist_3 = {'01': 0.4999999999999999, '10': 0.4999999999999999}



In [ ]:
from collections import OrderedDict
import functools
import numpy as np

In [ ]:
def plot_histogram(
    data,
    figsize=(7, 5),
    color=None,
    number_to_keep=None,
    sort="asc",
    target_string=None,
    legend=None,
    bar_labels=True,
    title=None,
    ax=None,
    filename=None,
):

    if not isinstance(data, list):
        data = [data]

    kind = "counts"
    for dat in data:
        if isinstance(dat, (QuasiDistribution, ProbDistribution)) or isinstance(
            next(iter(dat.values())), float
        ):
            kind = "distribution"
    return _plotting_core(
        data,
        figsize,
        color,
        number_to_keep,
        sort,
        target_string,
        legend,
        bar_labels,
        title,
        ax,
        filename,
        kind=kind,
    )


In [ ]:

def matplotlib_close_if_inline(figure):
    import matplotlib.pyplot

    MATPLOTLIB_INLINE_BACKENDS = {
        "module://ipykernel.pylab.backend_inline",
        "module://matplotlib_inline.backend_inline",
        "nbAgg",
    }

    if matplotlib.get_backend() in MATPLOTLIB_INLINE_BACKENDS:
        matplotlib.pyplot.close(figure)

In [ ]:

def _plotting_core(
    data,
    figsize=(7, 5),
    color=None,
    number_to_keep=None,
    sort="asc",
    target_string=None,
    legend=None,
    bar_labels=True,
    title=None,
    ax=None,
    filename=None,
    kind="counts",
):
    import matplotlib.pyplot as plt
    from matplotlib.ticker import MaxNLocator


    if isinstance(data, dict):
        data = [data]


    # Set bar colors
    if color is None:
        color = ["#648fff", "#dc267f", "#785ef0", "#ffb000", "#fe6100"]
    elif isinstance(color, str):
        color = [color]

    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig = None

    labels = sorted(functools.reduce(lambda x, y: x.union(y.keys()), data, set()))
    if number_to_keep is not None:
        labels.append("rest")



    length = len(data)
    width = 1 / (len(data) + 1)  # the width of the bars

    labels_dict, all_pvalues, all_inds = _plot_data(data, labels, number_to_keep, kind=kind)
    rects = []

    for item, _ in enumerate(data):
        label = None

        for idx, val in enumerate(all_pvalues[item]):
            if not idx and legend:
                label = legend[item]
            if val > 0:
                rects.append(
                    ax.bar(
                        idx + item * width,
                        val,
                        width,
                        label=label,
                        color=color[item % len(color)],
                        zorder=2,
                    )
                )
                label = None
        bar_center = (width / 2) * (length - 1)
        ax.set_xticks(all_inds[item] + bar_center)
        ax.set_xticklabels(labels_dict.keys(), fontsize=14, rotation=70)
        # attach some text labels
        if bar_labels:
            for rect in rects:
                for rec in rect:
                    height = rec.get_height()
                    if kind == "distribution":
                        height = round(height, 3)
                    if height >= 1e-3:
                        ax.text(
                            rec.get_x() + rec.get_width() / 2.0,
                            1.05 * height,
                            str(height),
                            ha="center",
                            va="bottom",
                            zorder=3,
                        )
                    else:
                        ax.text(
                            rec.get_x() + rec.get_width() / 2.0,
                            1.05 * height,
                            "0",
                            ha="center",
                            va="bottom",
                            zorder=3,
                        )
    #print(rects)
    # add some text for labels, title, and axes ticks
    if kind == "counts":
        ax.set_ylabel("Count", fontsize=14)
    else:
        ax.set_ylabel("Quasi-probability", fontsize=14)
    all_vals = np.concatenate(all_pvalues).ravel()
    min_ylim = 0.0
    if kind == "distribution":
        min_ylim = min(0.0, min(1.1 * val for val in all_vals))
    ax.set_ylim([min_ylim, min([1.1 * sum(all_vals), max(1.1 * val for val in all_vals)])])
    if "desc" in sort:
        ax.invert_xaxis()

    ax.yaxis.set_major_locator(MaxNLocator(5))
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(14)
    plt.grid(which="major", axis="y", zorder=0, linestyle="--")
    if title:
        plt.title(title)

    if legend:
        ax.legend(
            loc="upper left",
            bbox_to_anchor=(1.01, 1.0),
            ncol=1,
            borderaxespad=0,
            frameon=True,
            fontsize=12,
        )
    if fig:
        matplotlib_close_if_inline(fig)
    if filename is None:
        return fig
    else:
        return fig.savefig(filename)

In [ ]:
def _plot_data(data, labels, number_to_keep, kind="counts"):
    labels_dict = OrderedDict()
    all_pvalues = []
    all_inds = []

    if isinstance(data, dict):
        data = [data]
    #if number_to_keep is not None:
        #data = _unify_labels(_keep_largest_items(execution, number_to_keep) for execution in data)

    for execution in data:
        values = []
        for key in labels:
            if key not in execution:
                if number_to_keep is None:
                    labels_dict[key] = 1
                    values.append(0)
            else:
                labels_dict[key] = 1
                values.append(execution[key])
        if kind == "counts":
            pvalues = np.array(values, dtype=int)
        else:
            pvalues = np.array(values, dtype=float)
            pvalues /= np.sum(pvalues)
        all_pvalues.append(pvalues)
        numelem = len(values)
        ind = np.arange(numelem)  # the x locations for the groups
        all_inds.append(ind)
    return labels_dict, all_pvalues, all_inds

In [ ]:
counts = {'001': 0, '011': 30, '010': 0, '000': 0, '101': 0, '111': 0,
        '100': 0, '110': 0}

# Sort by the counts in descending order
plot_histogram(counts, sort='value_desc', legend=["1", "4"])


In [ ]:
plot_histogram([dist_1, dist_2, dist_3], legend=["1","2","3"])

In [ ]:
from qiskit.visualization import plot_distribution
plot_distribution([dist_1, dist_2, dist_3], legend=["1","2","3"])


In [ ]:
from qiskit.visualization import plot_histogram as ph
dist_1 = {'01': 0.369, '00': 0.13975, '10': 0.45875, '11': 0.0325}
dist_2 = {'00': 0.011835180722095105,
 '01': 0.48784140401270437,
 '10': 0.4983486948083151,
 '11': 0.001974720456885419}
dist_3 = {'01': 0.4999999999999999, '10': 0.4999999999999999, '00':0.000000000001}

legends = ["1", "2", "3"]
p = ph([dist_1, dist_2, dist_3], legend=legends)
len(p._localaxes[0].legend_.texts)


In [ ]:
from qiskit.visualization import plot_distribution
dist_1 = {'0': 0.369, '1': 0.13975}
dist_2 = {'0': 0, '1': 0.48784140401270437}
legends = ["1", "2"]
display(ph([dist_1, dist_2], legend=legends))
display(plot_distribution([dist_1, dist_2], legend=legends))